In [1]:
# importing dependencies
import pandas as pd
from sqlalchemy import create_engine
# need this later when creating sql database
import config

In [2]:
# read in the dataset and put into a dataframe
chicago_crime_file = "chicago_crime_stats.csv"
chicago_crime_df = pd.read_csv(chicago_crime_file)
# looking at the dataframe
chicago_crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12260346,JE102126,1/3/21 13:23,070XX S EGGLESTON AVE,486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,6.0,68,08B,1174496.0,1858251.0,2021,1/16/21 15:49,41.766435,-87.635964,"(41.766435144, -87.635963997)"
1,12263464,JE105797,1/3/21 6:59,080XX S YALE AVE,820,THEFT,$500 AND UNDER,RESIDENCE,False,False,...,17.0,44,6,1176011.0,1851718.0,2021,1/16/21 15:49,41.748474,-87.630607,"(41.748473982, -87.630606588)"
2,12259990,JE101773,1/3/21 0:20,056XX W WASHINGTON BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,29.0,25,08B,1138722.0,1900183.0,2021,1/16/21 15:49,41.882224,-87.766076,"(41.88222427, -87.766076162)"
3,12260669,JE102509,1/3/21 20:47,057XX S RACINE AVE,2022,NARCOTICS,POSSESS - COCAINE,STREET,True,False,...,16.0,67,18,1169298.0,1866822.0,2021,1/16/21 15:49,41.790069,-87.654769,"(41.79006908, -87.654768679)"
4,25702,JE102438,1/3/21 20:09,068XX S STONY ISLAND AVE,110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,5.0,43,01A,1188038.0,1860051.0,2021,1/10/21 15:51,41.771062,-87.586271,"(41.771062488, -87.586270811)"


# Transform
Here we take our dataframe and drop the columns we do not want included. 

In [3]:
# columns we want to keep and include in the new 
chicago_crime_cols = ["Date", "Primary Type", "Description", "Arrest", "Domestic", "District", "Year", "Latitude",
                     "Longitude"]

# creating a new dataframe of the columns we want to include in our database
chicago_crime_stats_df = chicago_crime_df[chicago_crime_cols].copy()

In [4]:
# looking at our new dataframe
chicago_crime_stats_df.head()

,Date,Primary Type,Description,Arrest,Domestic,District,Year,Latitude,Longitude
0,1/3/21 13:23,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,7,2021,41.766435,-87.635964
1,1/3/21 6:59,THEFT,$500 AND UNDER,False,False,6,2021,41.748474,-87.630607
2,1/3/21 0:20,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,15,2021,41.882224,-87.766076
3,1/3/21 20:47,NARCOTICS,POSSESS - COCAINE,True,False,7,2021,41.790069,-87.654769
4,1/3/21 20:09,HOMICIDE,FIRST DEGREE MURDER,False,False,3,2021,41.771062,-87.586271


# Limit the dataframe to violent crimes only

In [5]:
# take only those Primary Type's that are considered violent crimes

In [6]:
# checking the length of the dataframe
len(chicago_crime_stats_df)

53245

In [7]:
# limit dataframe to homicide, battery, assault, robbery, and sex offense
violent_crimes_df = chicago_crime_stats_df.loc[(chicago_crime_stats_df["Primary Type"]=="HOMICIDE") | 
                                               (chicago_crime_stats_df["Primary Type"]=="BATTERY") |
                                               (chicago_crime_stats_df["Primary Type"]=="ASSAULT") |
                                               (chicago_crime_stats_df["Primary Type"]=="ROBBERY") |
                                               (chicago_crime_stats_df["Primary Type"]=="SEX OFFENSE")]

# rename primary type as Primary_Type
violent_crimes_df = violent_crimes_df.rename(columns={"Primary Type": "Primary_Type"})

# looking at violent crimes only
violent_crimes_df.head()

,Date,Primary_Type,Description,Arrest,Domestic,District,Year,Latitude,Longitude
0,1/3/21 13:23,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,7,2021,41.766435,-87.635964
2,1/3/21 0:20,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,15,2021,41.882224,-87.766076
4,1/3/21 20:09,HOMICIDE,FIRST DEGREE MURDER,False,False,3,2021,41.771062,-87.586271
5,1/3/21 8:54,ASSAULT,SIMPLE,False,False,4,2021,41.702154,-87.563980
10,1/3/21 21:28,HOMICIDE,FIRST DEGREE MURDER,False,False,4,2021,41.736660,-87.544454


In [8]:
# format the date in the first column using datetime
# first need to take the time out of the row
violent_crimes_df["Date"] = violent_crimes_df["Date"].str.split(" ", 1, expand=True)

# now format the date using datetime
violent_crimes_df["Date"] = pd.to_datetime(violent_crimes_df["Date"],format="%m/%d/%y")

violent_crimes_df.head()

,Date,Primary_Type,Description,Arrest,Domestic,District,Year,Latitude,Longitude
0,2021-01-03,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,7,2021,41.766435,-87.635964
2,2021-01-03,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,15,2021,41.882224,-87.766076
4,2021-01-03,HOMICIDE,FIRST DEGREE MURDER,False,False,3,2021,41.771062,-87.586271
5,2021-01-03,ASSAULT,SIMPLE,False,False,4,2021,41.702154,-87.563980
10,2021-01-03,HOMICIDE,FIRST DEGREE MURDER,False,False,4,2021,41.736660,-87.544454


In [9]:
# set date as the index for SQL database
violent_crimes_df.set_index("Date", inplace=True)
violent_crimes_df.head()

,Primary_Type,Description,Arrest,Domestic,District,Year,Latitude,Longitude
Date,,,,,,,,
2021-01-03,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,7,2021,41.766435,-87.635964
2021-01-03,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,15,2021,41.882224,-87.766076
2021-01-03,HOMICIDE,FIRST DEGREE MURDER,False,False,3,2021,41.771062,-87.586271
2021-01-03,ASSAULT,SIMPLE,False,False,4,2021,41.702154,-87.563980
2021-01-03,HOMICIDE,FIRST DEGREE MURDER,False,False,4,2021,41.736660,-87.544454


In [10]:
# checking the length of the new dataframe
len(violent_crimes_df)

17782

In [11]:
# checking the data types
print(violent_crimes_df.dtypes)

Primary_Type     object
Description      object
Arrest             bool
Domestic           bool
District          int64
Year              int64
Latitude        float64
Longitude       float64
dtype: object


# Load the data into SQL Database

In [12]:
# Create a SQL Database connection
connection_string = f"postgres:{config.password}@localhost:5432/chicago_violents_crimes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
# checking tables
engine.table_names()

['violent_crimes_db']

In [14]:
# load dataframes into database
violent_crimes_df.to_sql(name="violent_crimes_db", con=engine, if_exists="append", index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Date" of relation "violent_crimes_db" does not exist
LINE 1: INSERT INTO violent_crimes_db ("Date", "Primary_Type", "Desc...
                                       ^

[SQL: INSERT INTO violent_crimes_db ("Date", "Primary_Type", "Description", "Arrest", "Domestic", "District", "Year", "Latitude", "Longitude") VALUES (%(Date)s, %(Primary_Type)s, %(Description)s, %(Arrest)s, %(Domestic)s, %(District)s, %(Year)s, %(Latitude)s, %(Longitude)s)]
[parameters: ({'Date': datetime.datetime(2021, 1, 3, 0, 0), 'Primary_Type': 'BATTERY', 'Description': 'DOMESTIC BATTERY SIMPLE', 'Arrest': False, 'Domestic': True, 'District': 7, 'Year': 2021, 'Latitude': 41.76643514, 'Longitude': -87.635964}, {'Date': datetime.datetime(2021, 1, 3, 0, 0), 'Primary_Type': 'BATTERY', 'Description': 'DOMESTIC BATTERY SIMPLE', 'Arrest': False, 'Domestic': True, 'District': 15, 'Year': 2021, 'Latitude': 41.88222427, 'Longitude': -87.76607616}, {'Date': datetime.da